# Ibovespa prediction / Feature Selection

In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
%matplotlib inline

n_jobs = 15 # number of parallel jobs to run in parallel

## Loading Dataset

In [2]:
data = pd.read_csv('dataset2.csv', index_col=0)
data.index = data.index.astype('datetime64[ns]')
data = data.loc['2013-01-01':,:]
date_index = pd.date_range(start=data.index.min(), end=data.index.max(), freq='D')
data = data.reindex(date_index)

In [3]:
data['IBOV_Direction'] = [1 if x >= 0 else 0 for x in data['IBOV_Close'].diff()]

In [4]:
data.columns

Index(['IBOV_Open', 'IBOV_High', 'IBOV_Low', 'IBOV_Close', 'IBOV_Volume',
       'Nasdaq_Close', 'Ouro_Close', 'S&P500_Close', 'Petróleo_Close',
       'Dow_Jones_Close', 'IBOV_Direction'],
      dtype='object')

In [5]:
lag_features = [
    'IBOV_Open',
    'IBOV_High',
    'IBOV_Low',
    'IBOV_Close',
    'IBOV_Volume',
    'IBOV_Direction'
]

wma_features = [
    'IBOV_Open',
    'IBOV_High',
    'IBOV_Low',
    'IBOV_Close',
    'IBOV_Volume',
    'Dow_Jones_Close',
    'Petróleo_Close',
    'S&P500_Close',
    'Nasdaq_Close',
    'Ouro_Close'
]

indicators_features = [
    'IBOV_Open',
    'IBOV_High',
    'IBOV_Low',
    'IBOV_Close'
]
    
to_concat = [data]

for f in lag_features:
    for i in range(1, 8):
        to_concat.append(
            data[[f]].shift(periods=i).rename(columns={f: f'{f}_L{i}'})
        )
        
for feature in wma_features:    
    to_concat.append(
        data[[feature]].shift(periods=1).rolling(
            window=30,
            center=False
        )
        .apply(lambda x: np.sum(np.arange(1, 31) * x) / np.sum(np.arange(1, 31)), raw=False)
        .rename(columns={feature: f'{feature}_WMA30'})
    )
    
#for feature in indicators_features:
#    to_concat.append(
#        (data[[feature]].shift(periods=1)
#        .ewm(span=12).mean() - data[[feature]].shift(periods=1)
#        .ewm(span=26).mean())
#        .rename(columns={feature: f'{feature}_MACD'})
#    )
    
data = pd.concat(to_concat, axis=1, join='inner')
data = data.iloc[30:,:]

to_drop = wma_features.copy()
to_drop.remove('IBOV_Close')
data.drop(to_drop, axis=1, inplace=True)

## Checking for empty values

In [6]:
(data.isnull().sum() > 0).sum()

0

There are no empty values in the dataset

In [7]:
data.columns

Index(['IBOV_Close', 'IBOV_Direction', 'IBOV_Open_L1', 'IBOV_Open_L2',
       'IBOV_Open_L3', 'IBOV_Open_L4', 'IBOV_Open_L5', 'IBOV_Open_L6',
       'IBOV_Open_L7', 'IBOV_High_L1', 'IBOV_High_L2', 'IBOV_High_L3',
       'IBOV_High_L4', 'IBOV_High_L5', 'IBOV_High_L6', 'IBOV_High_L7',
       'IBOV_Low_L1', 'IBOV_Low_L2', 'IBOV_Low_L3', 'IBOV_Low_L4',
       'IBOV_Low_L5', 'IBOV_Low_L6', 'IBOV_Low_L7', 'IBOV_Close_L1',
       'IBOV_Close_L2', 'IBOV_Close_L3', 'IBOV_Close_L4', 'IBOV_Close_L5',
       'IBOV_Close_L6', 'IBOV_Close_L7', 'IBOV_Volume_L1', 'IBOV_Volume_L2',
       'IBOV_Volume_L3', 'IBOV_Volume_L4', 'IBOV_Volume_L5', 'IBOV_Volume_L6',
       'IBOV_Volume_L7', 'IBOV_Direction_L1', 'IBOV_Direction_L2',
       'IBOV_Direction_L3', 'IBOV_Direction_L4', 'IBOV_Direction_L5',
       'IBOV_Direction_L6', 'IBOV_Direction_L7', 'IBOV_Open_WMA30',
       'IBOV_High_WMA30', 'IBOV_Low_WMA30', 'IBOV_Close_WMA30',
       'IBOV_Volume_WMA30', 'Dow_Jones_Close_WMA30', 'Petróleo_Close_WMA30',
   

## Datasets

### Classification

In [8]:
target = 'IBOV_Direction'
X_c = data.drop([target, 'IBOV_Close'], axis=1).values
y_c = data[target].values

### Regression

In [9]:
target = 'IBOV_Close'
X_r = data.drop([target, 'IBOV_Direction'], axis=1).values
y_r = data[target].values

# Feature Selection

## Genetic Filter

In [10]:
import pygad
from sklearn.feature_selection import mutual_info_regression
from scipy.stats import pearsonr
from sklearn.feature_selection import f_regression

def genetic_filter(X, y):
    def f(i, j):
        mi = mutual_info_regression(i.reshape(-1,1), j)[0]
        F, _ = f_regression(i.reshape(-1, 1), j)
        corr, _ = pearsonr(i, j)

        return mi + F[0] + abs(corr)

    f_values = {}
    ncols = X.shape[1]

    results = Parallel(n_jobs=n_jobs)(delayed(f)(X[:,i], X[:,j]) for i in range(ncols - 1) for j in range(i + 1, ncols))

    index = 0
    for i in range(ncols - 1):
        for j in range(i + 1, ncols):
            f_values[(i, j)] = results[index]
            f_values[(j, i)] = results[index]
            index += 1

    results = Parallel(n_jobs=n_jobs)(delayed(f)(y, X[:,i]) for i in range(ncols))
    index = 0
    for i in range(ncols):
        f_values[('target', i)] = results[index]
        index += 1
        
    def fitness_func(solution, solution_idx):
        idx_selected = np.nonzero(solution)[0]

        f_features_target = 0
        for idx in idx_selected:
            f_features_target += f_values[('target', idx)]

        f_features = 0
        for i in range(len(idx_selected) - 1):
            for j in range(i + 1, len(idx_selected)):
                f_features += f_values[(idx_selected[i], idx_selected[j])]

        return f_features_target - f_features

    ga = pygad.GA(
        num_parents_mating=4,
        keep_parents=3,
        sol_per_pop=100,
        num_generations=1000,
        num_genes=ncols,
        crossover_type='two_points',
        mutation_type='random',
        mutation_probability=0.001,
        parent_selection_type='rws',
        gene_space=(0, 1),
        fitness_func=fitness_func,
        parallel_processing=['thread', 15],
        stop_criteria=["saturate_15"]
    )

    ga.run()
    
    return ga

In [11]:
c_filter = np.nonzero(genetic_filter(X_c, y_c).best_solution()[0])[0]
X_c_filter = X_c[:, c_filter]

In [12]:
r_filter = np.nonzero(genetic_filter(X_r, y_r).best_solution()[0])[0]
X_r_filter = X_r[:, r_filter]

## PCA

In [13]:
from sklearn.decomposition import PCA

pca = PCA(n_components=20)

In [14]:
X_c_pca = pca.fit_transform(X_c, y_c)

In [15]:
X_r_pca = pca.fit_transform(X_r, y_r)

# Model training/validation

## Train/Test split function definition

In [16]:
def train_test_split(X, y, train_size=0.8):
    nrows = X.shape[0]
    sep = math.ceil(nrows * train_size)
    
    return X[:sep,:], y[:sep], X[sep:,:], y[sep:]

## SVM

In [17]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, accuracy_score, roc_curve, auc
from sklearn.preprocessing import MinMaxScaler

def run_svm(X_train_, y_train_, X_test_, y_test_):
    pipe = Pipeline([
        ('scaler', MinMaxScaler()),
        ('SVM', SVC(
            kernel='poly',
            degree=1
        ))
    ])

    pipe.fit(X_train_, y_train_)
    y_pred = pipe.predict(X_test_)
    
    fpr, tpr, _ = roc_curve(y_test_, y_pred)
    return auc(fpr, tpr), accuracy_score(y_test_, y_pred)

### All Features

In [18]:
X_train, y_train, X_test, y_test = train_test_split(X_c, y_c)

results = Parallel(n_jobs=n_jobs)(delayed(run_svm)(X_train, y_train, X_test, y_test) for i in range(50))
aucs, accs = zip(*results)
print(f'SVM AUC: {np.mean(aucs)} +- {np.std(aucs)}')
print(f'SVM Accuracy: {np.mean(accs)} +- {np.std(accs)}')

SVM AUC: 0.665085896179422 +- 1.1102230246251565e-16
SVM Accuracy: 0.6661931818181818 +- 0.0


### Genetic Filter Selected

In [19]:
X_train, y_train, X_test, y_test = train_test_split(X_c_filter, y_c)

results = Parallel(n_jobs=n_jobs)(delayed(run_svm)(X_train, y_train, X_test, y_test) for i in range(50))
aucs, accs = zip(*results)
print(f'SVM AUC: {np.mean(aucs)} +- {np.std(aucs)}')
print(f'SVM Accuracy: {np.mean(accs)} +- {np.std(accs)}')

SVM AUC: 0.665085896179422 +- 1.1102230246251565e-16
SVM Accuracy: 0.6661931818181818 +- 0.0


In [20]:
X_train.shape

(2816, 20)

### PCA Selected

In [21]:
X_train, y_train, X_test, y_test = train_test_split(X_c_pca, y_c)

results = Parallel(n_jobs=n_jobs)(delayed(run_svm)(X_train, y_train, X_test, y_test) for i in range(50))
aucs, accs = zip(*results)
print(f'SVM AUC: {np.mean(aucs)} +- {np.std(aucs)}')
print(f'SVM Accuracy: {np.mean(accs)} +- {np.std(accs)}')

SVM AUC: 0.5060746944428695 +- 1.1102230246251565e-16
SVM Accuracy: 0.5213068181818182 +- 0.0


## ANN - MLP

In [22]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, roc_curve, auc
from itertools import combinations_with_replacement
from joblib import Parallel, delayed

n_layers = np.arange(2) + 1
n_neurons = np.arange(0, 35, 5) + 5
epochs = [500]
combinations = []

for layers in n_layers:
    combinations.extend(combinations_with_replacement(n_neurons, int(layers)))
    
def run_mlp(X_train, y_train, X_test, y_test, layer_sizes, epochs):
    import warnings
    warnings.filterwarnings("ignore")
    
    aucs = []
    accs = []
    
    for i in range(50):
        pipe = Pipeline([
            ('scaler', MinMaxScaler()),
            ('MLP', MLPClassifier(
                hidden_layer_sizes=layer_sizes,
                solver='adam',
                max_iter=epochs,
                activation='tanh'
            ))
        ])

        pipe.fit(X_train, y_train)
        y_pred = pipe.predict(X_test)
        fpr, tpr, _ = roc_curve(y_test, y_pred)
        aucs.append(auc(fpr, tpr))
        accs.append(accuracy_score(y_test, y_pred))
    
    return epochs, layer_sizes, aucs, accs

### All Features

In [23]:
X_train, y_train, X_test, y_test = train_test_split(X_c, y_c)

results = Parallel(n_jobs=n_jobs)(delayed(run_mlp)(
    X_train,
    y_train,
    X_test,
    y_test,
    ls,
    ep
) for ep in epochs for ls in combinations)

data = []
for result in results:
    data.append([
        result[0],
        result[1],
        f'{np.mean(result[2])} +- {np.std(result[2])}',
        f'{np.mean(result[3])} +- {np.std(result[3])}'
    ])
    
summary = pd.DataFrame(data, columns=['Epochs', 'Hidden Layer Sizes', 'AUC', 'Accuracy'])
summary.sort_values(['Accuracy', 'AUC'], ascending=False)

,Epochs,Hidden Layer Sizes,AUC,Accuracy
4,500,"(25,)",0.6585373755700088 +- 0.006332192436238034,0.6590340909090908 +- 0.005759133062915251
3,500,"(20,)",0.6580828264338304 +- 0.0057688905467205434,0.6584659090909091 +- 0.0057112872150586625
5,500,"(30,)",0.6569878425115216 +- 0.007447246986823078,0.6572443181818182 +- 0.007571169301838877
2,500,"(15,)",0.6561846869102484 +- 0.008104827706845397,0.6567897727272727 +- 0.007606477723456365
1,500,"(10,)",0.6562707855794854 +- 0.007134811739289943,0.6564772727272729 +- 0.008107100435996757
6,500,"(35,)",0.6547221434761832 +- 0.010436132283908902,0.655340909090909 +- 0.009274426092154866
18,500,"(10, 30)",0.6537501113693982 +- 0.011157400681731792,0.6548011363636363 +- 0.00997353322186706
9,500,"(5, 15)",0.6535651166746312 +- 0.008539467853996459,0.6541761363636364 +- 0.008880783603208943
12,500,"(5, 30)",0.6531551152977005 +- 0.010706512128706004,0.6540625 +- 0.010630505683395812
8,500,"(5, 10)",0.6525860379222925 +- 0.01042662294550057,0.6538920454545455 +- 0.010062634745774063


### Genetic Filter Selected

In [24]:
X_train, y_train, X_test, y_test = train_test_split(X_c_filter, y_c)

results = Parallel(n_jobs=n_jobs)(delayed(run_mlp)(
    X_train,
    y_train,
    X_test,
    y_test,
    ls,
    ep
) for ep in epochs for ls in combinations)

data = []
for result in results:
    data.append([
        result[0],
        result[1],
        f'{np.mean(result[2])} +- {np.std(result[2])}',
        f'{np.mean(result[3])} +- {np.std(result[3])}'
    ])
    
summary = pd.DataFrame(data, columns=['Epochs', 'Hidden Layer Sizes', 'AUC', 'Accuracy'])
summary.sort_values(['Accuracy', 'AUC'], ascending=False)

,Epochs,Hidden Layer Sizes,AUC,Accuracy
1,500,"(10,)",0.6554861780452442 +- 0.011075974815731669,0.6575852272727274 +- 0.009381067251593794
3,500,"(20,)",0.6536114463442488 +- 0.010148452779596797,0.6559375 +- 0.009061074980581443
2,500,"(15,)",0.6528031070037177 +- 0.010953294422537379,0.6553693181818183 +- 0.00953669951581566
12,500,"(5, 30)",0.6532081676291682 +- 0.012707271590404457,0.6551136363636363 +- 0.011525831122216347
5,500,"(30,)",0.6524529616160308 +- 0.011182066995325101,0.6551136363636363 +- 0.0098206738735475
18,500,"(10, 30)",0.6514900010529471 +- 0.012350119335458978,0.6541761363636364 +- 0.010542835257095904
4,500,"(25,)",0.6501274875873745 +- 0.010200575840195148,0.6528693181818183 +- 0.009028058663603766
13,500,"(5, 35)",0.6497607380348768 +- 0.01685118709806503,0.6525852272727272 +- 0.01432925370080035
7,500,"(5, 5)",0.6499516454322348 +- 0.014323445627947122,0.6525568181818181 +- 0.012136368956416937
10,500,"(5, 20)",0.6502386139977159 +- 0.01455043608835144,0.6524715909090909 +- 0.013581833441712847


### PCA Selected

In [25]:
X_train, y_train, X_test, y_test = train_test_split(X_c_pca, y_c)

results = Parallel(n_jobs=n_jobs)(delayed(run_mlp)(
    X_train,
    y_train,
    X_test,
    y_test,
    ls,
    ep
) for ep in epochs for ls in combinations)

data = []
for result in results:
    data.append([
        result[0],
        result[1],
        f'{np.mean(result[2])} +- {np.std(result[2])}',
        f'{np.mean(result[3])} +- {np.std(result[3])}'
    ])
    
summary = pd.DataFrame(data, columns=['Epochs', 'Hidden Layer Sizes', 'AUC', 'Accuracy'])
summary.sort_values(['Accuracy', 'AUC'], ascending=False)

,Epochs,Hidden Layer Sizes,AUC,Accuracy
26,500,"(20, 25)",0.5045185197184581 +- 0.00732332348228089,0.5185511363636365 +- 0.009855170552654112
5,500,"(30,)",0.5039474984408284 +- 0.008109091034802453,0.5180397727272728 +- 0.010623062839936494
27,500,"(20, 30)",0.5031847598065817 +- 0.007702119795581226,0.5176136363636363 +- 0.010568792748714927
28,500,"(20, 35)",0.5034429748183666 +- 0.010000573969652827,0.5171590909090908 +- 0.013765487145889039
31,500,"(25, 35)",0.5031759312506581 +- 0.008020065963378939,0.5165625 +- 0.011340921900612191
29,500,"(25, 25)",0.5029102646136899 +- 0.009809925310297049,0.5161079545454546 +- 0.01395696163750644
34,500,"(35, 35)",0.5030369422418053 +- 0.008537548118288031,0.5160511363636364 +- 0.012465247403090242
22,500,"(15, 25)",0.5031850837902853 +- 0.008390842254596116,0.5156818181818182 +- 0.011180917369480035
23,500,"(15, 30)",0.501959453439492 +- 0.010772195373096993,0.5155113636363636 +- 0.014370732388440235
33,500,"(30, 35)",0.5025670038797049 +- 0.00807563809940298,0.5153693181818181 +- 0.011489677421134825


# Ensemble A

In [26]:
import warnings
warnings.filterwarnings("ignore")
from joblib import Parallel, delayed
from sklearn.tree import DecisionTreeClassifier
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input, SimpleRNN
from itertools import combinations_with_replacement

class EnsembleA():
    def __init__(self, n_features, neurons, epochs=500):
        self.regressor = Sequential()
        self.regressor.add(SimpleRNN(neurons, activation='tanh', input_shape=(1, n_features)))
        self.regressor.add(Dense(1, activation='linear'))
        self.regressor.compile(loss='mean_squared_error', optimizer='adam')
        
        self.tree_classifier = DecisionTreeClassifier()

        self.epochs = epochs
            
    def fit(self, X, y):
        _X = X.reshape((X.shape[0], 1, X.shape[1]))
        self.regressor.fit(_X, y, epochs=self.epochs, verbose=0)
        y_reg_pred = self.regressor.predict(_X, verbose=0)
        self.tree_classifier.fit(y_reg_pred.reshape(-1, 1), y)
        
    def predict(self, X):
        _X = X.reshape((X.shape[0], 1, X.shape[1]))
        y_pred = self.regressor.predict(_X, verbose=0)
        return self.tree_classifier.predict(y_pred.reshape(-1, 1))
    
n_neurons = np.arange(0, 35, 5) + 5
epochs = [500]
    
def run_ensemble_a(X_train, y_train, X_test, y_test, neurons, epochs):
    import warnings
    import os
    warnings.filterwarnings("ignore")
    
    aucs = []
    accs = []
    
    for i in range(50):
        model = EnsembleA(X_train.shape[1], neurons, epochs)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        fpr, tpr, _ = roc_curve(y_test, y_pred)
        aucs.append(auc(fpr, tpr))
        accs.append(accuracy_score(y_test, y_pred))
    
    return epochs, neurons, aucs, accs

In [27]:
X_train, y_train, X_test, y_test = train_test_split(X_c, y_c)

results = Parallel(n_jobs=n_jobs)(delayed(run_ensemble_a)(X_train, y_train, X_test, y_test, n, e) for n in n_neurons for e in epochs)
epochs, neurons, aucs, accs = zip(*results)
summary = pd.DataFrame(data, columns=['Epochs', 'Neurons', 'AUC', 'Accuracy'])
summary.sort_values(['Accuracy', 'AUC'], ascending=False)

,Epochs,Neurons,AUC,Accuracy
26,500,"(20, 25)",0.5045185197184581 +- 0.00732332348228089,0.5185511363636365 +- 0.009855170552654112
5,500,"(30,)",0.5039474984408284 +- 0.008109091034802453,0.5180397727272728 +- 0.010623062839936494
27,500,"(20, 30)",0.5031847598065817 +- 0.007702119795581226,0.5176136363636363 +- 0.010568792748714927
28,500,"(20, 35)",0.5034429748183666 +- 0.010000573969652827,0.5171590909090908 +- 0.013765487145889039
31,500,"(25, 35)",0.5031759312506581 +- 0.008020065963378939,0.5165625 +- 0.011340921900612191
29,500,"(25, 25)",0.5029102646136899 +- 0.009809925310297049,0.5161079545454546 +- 0.01395696163750644
34,500,"(35, 35)",0.5030369422418053 +- 0.008537548118288031,0.5160511363636364 +- 0.012465247403090242
22,500,"(15, 25)",0.5031850837902853 +- 0.008390842254596116,0.5156818181818182 +- 0.011180917369480035
23,500,"(15, 30)",0.501959453439492 +- 0.010772195373096993,0.5155113636363636 +- 0.014370732388440235
33,500,"(30, 35)",0.5025670038797049 +- 0.00807563809940298,0.5153693181818181 +- 0.011489677421134825


# Ensemble B

In [28]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

class EnsembleB():
    def __init__(self, n_features, neurons, epochs=500):
        self.kmeans = KMeans(n_clusters=2)
        self.classifier = EnsembleA(n_features+1, neurons, epochs)
        
    def fit(self, X, y):
        scaler = MinMaxScaler()
        _X = scaler.fit_transform(X)
        self.kmeans.fit(_X)
        _X = np.append(X, self.kmeans.labels_.reshape(-1, 1), axis=1)
        self.classifier.fit(_X, y)
        
    def predict(self, X):
        _X = np.append(X, self.kmeans.predict(X).reshape(-1, 1), axis=1)
        return self.classifier.predict(_X)
    
n_neurons = np.arange(0, 35, 5) + 5
epochs = [500]
    
def run_ensemble_b(X_train, y_train, X_test, y_test, neurons, epochs):
    import warnings
    warnings.filterwarnings("ignore")
    
    aucs = []
    accs = []
    
    for i in range(50):
        model = EnsembleB(X_train.shape[1], neurons, epochs)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        fpr, tpr, _ = roc_curve(y_test, y_pred)
        aucs.append(auc(fpr, tpr))
        accs.append(accuracy_score(y_test, y_pred))
    
    return epochs, neurons, aucs, accs

In [29]:
X_train, y_train, X_test, y_test = train_test_split(X_c, y_c)

results = Parallel(n_jobs=n_jobs)(delayed(run_ensemble_b)(X_train, y_train, X_test, y_test, n, e) for n in n_neurons for e in epochs)
epochs, neurons, aucs, accs = zip(*results)
summary = pd.DataFrame(data, columns=['Epochs', 'Neurons', 'AUC', 'Accuracy'])
summary.sort_values(['Accuracy', 'AUC'], ascending=False)

,Epochs,Neurons,AUC,Accuracy
26,500,"(20, 25)",0.5045185197184581 +- 0.00732332348228089,0.5185511363636365 +- 0.009855170552654112
5,500,"(30,)",0.5039474984408284 +- 0.008109091034802453,0.5180397727272728 +- 0.010623062839936494
27,500,"(20, 30)",0.5031847598065817 +- 0.007702119795581226,0.5176136363636363 +- 0.010568792748714927
28,500,"(20, 35)",0.5034429748183666 +- 0.010000573969652827,0.5171590909090908 +- 0.013765487145889039
31,500,"(25, 35)",0.5031759312506581 +- 0.008020065963378939,0.5165625 +- 0.011340921900612191
29,500,"(25, 25)",0.5029102646136899 +- 0.009809925310297049,0.5161079545454546 +- 0.01395696163750644
34,500,"(35, 35)",0.5030369422418053 +- 0.008537548118288031,0.5160511363636364 +- 0.012465247403090242
22,500,"(15, 25)",0.5031850837902853 +- 0.008390842254596116,0.5156818181818182 +- 0.011180917369480035
23,500,"(15, 30)",0.501959453439492 +- 0.010772195373096993,0.5155113636363636 +- 0.014370732388440235
33,500,"(30, 35)",0.5025670038797049 +- 0.00807563809940298,0.5153693181818181 +- 0.011489677421134825
